# Importação de bibliotecas 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU, Conv1D, MaxPooling1D,  Bidirectional
from sklearn.model_selection import cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
import time
import os
import warnings
warnings.filterwarnings("ignore")
import statistics
import statsmodels.api as sm 
from PyEMD import EMD, EEMD, CEEMDAN

# Pasta indicada como raiz 
* O comando os.listdir retorna uma lista com o nome de todos os arquivos contidos na pasta

In [ ]:
file = r'BASE_DADOS_MET'
arquivos = os.listdir(file)

# Montar Janela decomposição Sazonal

In [ ]:
def montarJ(data, janela, step,  trend, sazo, ruido):
    
    x, y = [],[]
    
    for i in range(janela, len(data)):
        
        fim_y = i+step
        if fim_y > len(data):
            break
        a = np.array(data[i-janela:i,0])
        t = trend[i-1:i,0]
        s = sazo[i-1:i,0]
        r = ruido[i-1:i,0]
        
        c = np.concatenate((a, t, s, r))
        
        
        x.append(c)
        
        y.append(data[i:fim_y,0])
        
    return np.array(x), np.array(y)

# Montar janela decomposição EEMD

In [ ]:
def montarJ1(data, janela, step, IMF1, IMF2, IMF3, IMF4, IMF5, res1):
    
    x, y = [],[]
    
    for i in range(janela, len(data)):
        
        fim_y = i+step
        if fim_y > len(data):
            break
        a = np.array(data[i-janela:i,0])
        IM1 = IMF1[i-1:i,0] 
        IM2 = IMF2[i-1:i,0] 
        IM3 = IMF3[i-1:i,0] 
        IM4 = IMF4[i-1:i,0] 
        IM5 = IMF5[i-1:i,0]
        rs1 = res1[i-1:i,0] 
        
        c = np.concatenate((a, IM1, IM2, IM3, IM4, IM5, rs1))
        
        
        x.append(c)
        
        y.append(data[i:fim_y,0])
        
    return np.array(x), np.array(y)

# Montar janela dados sem decomposição 

In [ ]:
def montarJanela(data, janela, step):
    
    x, y = [],[]
    
    for i in range(janela, len(data)):
        
        fim_y = i+step
        if fim_y > len(data):
            break
        
        x.append(data[i-janela:i,0])
        y.append(data[i:fim_y,0])
        
    return np.array(x), np.array(y)

# Avalia modelo

In [ ]:
def avaliaModelo1(y_test, resul, nomeModel, REGIAO):
    
    tm = len(y_test.columns)
    res = 0
    RMSE = 0
    EMA = 0
    cor = 0
    
    print("------------------------------//-------------------------------------")
    print(f"REGIAO - {REGIAO}")
    print(f"Resultados {nomeModel}")
    if(tm > 1):
        for i in range(len(y_test.columns)):
            res = res + (metrics.r2_score(y_test.iloc[:, i], resul.iloc[:, i]))
            cor = cor + (y_test.iloc[:, i].corr(resul.iloc[:, i]))
            RMSE = RMSE + (math.sqrt(metrics.mean_squared_error(y_test.iloc[:, i], resul.iloc[:, i])))
            EMA = EMA + (metrics.mean_absolute_error(y_test.iloc[:, i], resul.iloc[:, i]))
        
        print(f"{nomeModel} - R2 = {res/tm}")
        print(f"{nomeModel} - COR = {cor/tm}")
        print(f"{nomeModel} - Raiz Erro Medio Quad (REMQ) = {RMSE/tm}")
        print(f"{nomeModel} - Erro Medio Abs (EMA) = {EMA/tm}")
    else:
        print(f"{nomeModel} - R2 = {metrics.r2_score(y_test, resul)}")
        print(f"{nomeModel} - COR = {y_test.iloc[:, 0].corr(resul.iloc[:, 0])}")
        print(f"{nomeModel} - Raiz Erro Medio Quad (REMQ) = {math.sqrt(metrics.mean_squared_error(y_test, resul))}")
        print(f"{nomeModel} - Erro Medio Abs (EMA) = {metrics.mean_absolute_error(y_test, resul)}")

# Plota Gráficos 

In [ ]:
def plotGraficoPR(real, prev):
    tm = len(real.columns)
    if(tm > 1):
        fig, axs = plt.subplots(nrows = tm, figsize = (16, 12))
        for i in range(len(real.columns)):
            axs[i].plot(real.iloc[:, i], label = 'Dados Reais')
            axs[i].plot(prev.iloc[:, i], label = 'Dados Previstos')

    else:
        plt.figure(figsize=(16, 5))
        plt.plot(real, label = 'Dados Reais')
        plt.plot(prev, label = 'Dados Reais')
        #plt.legend()

# Cria RNA CNN-LSTM

In [ ]:
def criaRedeLSTM(x_train, y_train, x_test):
    
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    
    regressor = Sequential()
    regressor.add(Conv1D(filters = 128, kernel_size = 2, activation = 'tanh', input_shape=(x_train.shape[1], 1)))
    regressor.add(MaxPooling1D(pool_size = 1))
    regressor.add(LSTM(units = 200, return_sequences = True, activation = 'tanh'))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50, activation = 'tanh'))
    regressor.add(Dropout(0.1))
    regressor.add(Dense(units = 25, activation = 'linear'))
    regressor.add(Dropout(0.1))
    regressor.add(Dense(y_train.shape[1], activation = 'linear'))
    
    regressor.compile(loss= 'mean_squared_error', optimizer= 'Adam', metrics=['mean_absolute_error'])
    regressor.fit(x_train, y_train, epochs= 400, batch_size= 512, verbose=0)
    
    resul = regressor.predict(x_test)
    
    return resul

# Cria Random Forest

In [ ]:
def criaRDForest(x_train, y_train, x_test):
    
    regressor = RandomForestRegressor(n_estimators= 60, min_samples_split = 32, min_samples_leaf = 64, max_leaf_nodes = 72, max_depth = 9)
    regressor.fit(x_train, y_train)
    prev = regressor.predict(x_test)

    return prev

# Cria árvore decisão

In [ ]:
def criaARV(x_train, y_train, x_test):
    regressor = DecisionTreeRegressor(min_samples_split = 512, min_samples_leaf = 128, max_leaf_nodes = 24, max_depth = 24) 
    regressor.fit(x_train, y_train)
    prev = regressor.predict(x_test)

    return prev
    

# Aplica Modelos com a decomposição sazonal

In [ ]:
resultados = pd.DataFrame()

cont = 1
while (cont < len(arquivos)):
    
    #Monta o caminho com um arquivo específico 
    caminho = r""+file+"\\"+arquivos[cont]
    
    #Abre o arquivo na pasta 
    df = pd.read_excel(caminho)
    
    
    #atribui os dados a variável DTF
    DTF = df[['PRECIPITACAO(mm) (ACM)']]

    #VENTO_VEL_H (m/s) (MD)  PRECIPITACAO(mm) (ACM) UMID_REL_AR_H (%) (MD) TEMPMED (C) (MD)
    
    
    # Muda a escala dos valores para 0 a 1
    scaler = MinMaxScaler()
    dataF = scaler.fit_transform(DTF)
    
    
    """
    Aplica a decomposição sazonal
    uso de extrapolate_trend ajuda a garantir que a série de tendência, sazonalidade e ruído tenham o mesmo 
    tamanho que a série original.   extrapolate_trend = 5  método de decomposição tenta extrapolar a tendência 
    para além dos limites da série temporal, usando dados adicionais para suavizar o comportamento da tendência. 
    Com extrapolate_trend=5, por exemplo, a tendência é extrapolada usando 5 pontos de dados extras na borda da 
    série para suavização"""
    
    decomposicao = sm.tsa.seasonal_decompose(DTF, model='additive', period  = 3, extrapolate_trend = 5)
    
    #obtem tendencia, sazonalidade e o ruido 
    tendencia =  decomposicao.trend
    sazonalidade = decomposicao.seasonal
    ruido = decomposicao.resid
    
    print('Tamanho DTF: ',len(DTF))
    print('Tamanho Tende ',len(tendencia))
    print('Tamanho sazonalidade: ',len(sazonalidade))
    
    #Preenche a tendencia, sazonalidade de ruido com 0
    tendencia = np.nan_to_num(tendencia)
    sazonalidade = np.nan_to_num(sazonalidade)
    ruido = np.nan_to_num(ruido)
    
    #Normaliza a tendência a sozonalidade e o ruído
    trend = np.reshape(tendencia, (-1, 1))
    sazo = np.reshape(sazonalidade, (-1, 1))
    rd = np.reshape(ruido, (-1, 1))
    
    
    scaler1 = MinMaxScaler()
    trend = scaler1.fit_transform(trend)
    
    scaler2 = MinMaxScaler()
    sazo = scaler2.fit_transform(sazo)
    
    scaler3 = MinMaxScaler()
    rd = scaler3.fit_transform(rd)
    
    x, y = montarJ(dataF, 3, 1, trend, sazo, rd)
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size= 0.3, random_state= 0, shuffle=False)
    
    
    #regressor = RandomForestRegressor()
    PREV_LSTM = criaRedeLSTM(X_train, y_train, X_test)
    PREV_RF = criaRDForest(X_train, y_train, X_test)
    PREV_ARV = criaARV(X_train, y_train, X_test)
    
    
    
    #Quando a previsão for de um único dia a frente é necessário um reshape dos dados previstos para reverter a escala, quando for de mais de um dia não é necessário  
    
    PREV_LSTM = np.reshape(PREV_LSTM, (-1, 1))
    PREV_RF = np.reshape(PREV_RF, (-1, 1))
    PREV_ARV = np.reshape(PREV_ARV, (-1, 1))

    PREV_LSTM = scaler.inverse_transform(PREV_LSTM)
    PREV_RF = scaler.inverse_transform(PREV_RF)
    PREV_ARV = scaler.inverse_transform(PREV_ARV)
    
    
    Real = scaler.inverse_transform(y_test)
    
    
    #Somente quando for para prever mais de um passo a frente 
    Real = pd.DataFrame(Real)
    PREV_LSTM = pd.DataFrame(PREV_LSTM)
    PREV_RF = pd.DataFrame(PREV_RF)
    PREV_ARV = pd.DataFrame(PREV_ARV)
    
    
    avaliaModelo1(Real, PREV_LSTM, "DECOMP_SD_LSTM", arquivos[cont])
    avaliaModelo1(Real, PREV_RF, "DECOMP_SD_R_FOREST", arquivos[cont])
    avaliaModelo1(Real, PREV_ARV, "DECOMP_SD_ARVORE", arquivos[cont])
    
    resultados['DADOS_REAIS'] = Real[0]
    resultados['DC_SD_LSTM'] = PREV_LSTM[0]
    resultados['DC_SD_RF'] = PREV_RF[0]
    resultados['DC_SD_ARV'] = PREV_ARV[0]
  
    nome = 'resultados_DC_AD'+'_'+arquivos[cont]
    resultados.to_excel(nome)
    
    
    cont = cont+3

# Aplica Modelos com a decomposição EEMD

In [ ]:
resultados = pd.DataFrame()

cont = 2

while (cont < len(arquivos)):
    
    #Monta o caminho com um arquivo específico 
    caminho = r""+file+"\\"+arquivos[cont]
    
    #Abre o arquivo na pasta 
    df = pd.read_excel(caminho)
    
    
    #atribui os dados a variável DTF
    DTF = df['UMID_REL_AR_H (%) (MD)'].values
    #VENTO_VEL_H (m/s) (MD)  PRECIPITACAO(mm) (ACM) UMID_REL_AR_H (%) (MD) TEMPMED (C) (MD)
    
    DTF1 = np.reshape(DTF, (-1, 1))
    
    # Muda a escala dos valores para 0 a 1
    scaler = MinMaxScaler()
    dataF = scaler.fit_transform(DTF1)
    
    """Os dados decompostos também serão inseridos nas janelas, mas apenas os valores correspondentes ao ponto que se 
    encontra na localização final da janela ex.: dados normais tamanho da janela = 3, incluir na janela a tendência, 
    sazonalidade e ruído no ponto correstondente a 3 em sua janela, ou seja, um único dado para cada """
    
    t = np.linspace(0, 1, len(DTF))  # Tempo
    
    # Criar uma instância do EMD
    eemd = EEMD()

    # Realizar a decomposição
    imfs = eemd.eemd(DTF, t)
    
    #Obtem o resíduo
    res = DTF - np.sum(imfs, axis=0)

    # Limite a 5 IMFs
    imfs_limitadas = imfs[:5]

    imf1 = imfs[0]
    imf2 = imfs[1]
    imf3 = imfs[2]
    imf4 = imfs[3]
    imf5 = imfs[4]
    imf6 = imfs[5]
    
    #plt.plot(imf6)
    #plt.show()
    
    
    #Normaliza a tendência a sozonalidade e o ruído
    IMF1 = np.reshape(imf1, (-1, 1))
    IMF2 = np.reshape(imf2, (-1, 1))
    IMF3 = np.reshape(imf3, (-1, 1))
    IMF4 = np.reshape(imf4, (-1, 1))
    IMF5 = np.reshape(imf5, (-1, 1))
    #IMF6 = np.reshape(imf6, (-1, 1))
    
    # Aplica a normalização 
    scaler1 = MinMaxScaler()
    IMF1 = scaler1.fit_transform(IMF1)
    
    scaler2 = MinMaxScaler()
    IMF2 = scaler2.fit_transform(IMF2)
    
    scaler3 = MinMaxScaler()
    IMF3 = scaler3.fit_transform(IMF3)
    
    scaler4 = MinMaxScaler()
    IMF4 = scaler4.fit_transform(IMF4)
    
    scaler5 = MinMaxScaler()
    IMF5 = scaler5.fit_transform(IMF5)
    
    #scaler6 = MinMaxScaler()
    #IMF6 = scaler5.fit_transform(IMF6)
    
    res = np.reshape(res, (-1, 1))
    scaler5 = MinMaxScaler()
    res1 = scaler5.fit_transform(res)
    
    
    # Monta a janela e divide o conjunto de dados
    x, y = montarJ1(dataF, 3, 3, IMF1, IMF2, IMF3, IMF4, IMF5, res1)
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size= 0.3, random_state= 0)
    
    
    #Aplica os modelos de previsao
    PREV_LSTM = criaRedeLSTM(X_train, y_train, X_test)
    PREV_RF = criaRDForest(X_train, y_train, X_test)
    PREV_ARV = criaARV(X_train, y_train, X_test)
    
    
    #Quando a previsão for de um único dia a frente é necessário um reshape dos dados previstos para reverter a escala, quando for de mais de um dia não é necessário  
    #PREV_LSTM = np.reshape(PREV_LSTM, (-1, 1))
    #PREV_RF = np.reshape(PREV_RF, (-1, 1))
    #PREV_ARV = np.reshape(PREV_ARV, (-1, 1))

    
    PREV_LSTM = scaler.inverse_transform(PREV_LSTM)
    PREV_RF = scaler.inverse_transform(PREV_RF)
    PREV_ARV = scaler.inverse_transform(PREV_ARV)
    
    
    Real = scaler.inverse_transform(y_test)
    
    
    #Somente quando for para prever mais de um passo a frente 
    Real = pd.DataFrame(Real)
    PREV_LSTM = pd.DataFrame(PREV_LSTM)
    PREV_RF = pd.DataFrame(PREV_RF)
    PREV_ARV = pd.DataFrame(PREV_ARV)
    
    avaliaModelo1(Real, PREV_LSTM, "DECOMP_EEMD_LSTM", arquivos[cont])
    avaliaModelo1(Real, PREV_RF, "DECOMP_EEMD_R_FOREST", arquivos[cont])
    avaliaModelo1(Real, PREV_ARV, "DECOMP_EEMD_ARVORE", arquivos[cont])
    
    #resultados['DADOS_REAIS'] = Real[0]
    #resultados['DC_EEMD_LSTM'] = PREV_LSTM[0]
    #resultados['DC_EEMD_RF'] = PREV_RF[0]
    #resultados['DC_EEMD_ARV'] = PREV_ARV[0]
  
    #nome = 'resultados_DC_EEMD'+'_'+arquivos[cont]
    #resultados.to_excel(nome)
    
    cont = cont+3

# Aplica modelos sem decomposição 

In [ ]:
resultados = pd.DataFrame()
cont = 2

while (cont < len(arquivos)):
    
    #Monta o caminho com um arquivo específico 
    caminho = r""+file+"\\"+arquivos[cont]
    
    #Abre o arquivo na pasta 
    df = pd.read_excel(caminho)
    
    
    #atribui os dados a variável DTF
    DTF = df['PRECIPITACAO(mm) (ACM)'].values
    #VENTO_VEL_H (m/s) (MD)  PRECIPITACAO(mm) (ACM) UMID_REL_AR_H (%) (MD) TEMPMED (C) (MD)
    
    DTF1 = np.reshape(DTF, (-1, 1))
    
    # Muda a escala dos valores para 0 a 1
    scaler = MinMaxScaler()
    dataF = scaler.fit_transform(DTF1)
    
    # Montar Janela
    x, y = montarJanela(dataF, 3, 4)
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size= 0.3, random_state= 0)
    
    
    #regressor = RandomForestRegressor()
    PREV_LSTM = criaRedeLSTM(X_train, y_train, X_test)
    PREV_RF = criaRDForest(X_train, y_train, X_test)
    PREV_ARV = criaARV(X_train, y_train, X_test)
    
    
    #Quando a previsão for de um único dia a frente é necessário um reshape dos dados previstos para reverter a escala, quando for de mais de um dia não é necessário  
    #PREV_LSTM = np.reshape(PREV_LSTM, (-1, 1))
    #PREV_RF = np.reshape(PREV_RF, (-1, 1))
    #PREV_ARV = np.reshape(PREV_ARV, (-1, 1))

    
    PREV_LSTM = scaler.inverse_transform(PREV_LSTM)
    PREV_RF = scaler.inverse_transform(PREV_RF)
    PREV_ARV = scaler.inverse_transform(PREV_ARV)
    
    
    Real = scaler.inverse_transform(y_test)
    
    
    #Somente quando for para prever mais de um passo a frente 
    Real = pd.DataFrame(Real)
    PREV_LSTM = pd.DataFrame(PREV_LSTM)
    PREV_RF = pd.DataFrame(PREV_RF)
    PREV_ARV = pd.DataFrame(PREV_ARV)
    
    avaliaModelo1(Real, PREV_LSTM, "SEM_DECOMP_LSTM", arquivos[cont])
    avaliaModelo1(Real, PREV_RF, "SEM_DECOMP_R_FOREST", arquivos[cont])
    avaliaModelo1(Real, PREV_ARV, "SEM_DECOMP_ARVORE", arquivos[cont])
    
    #resultados['DADOS_REAIS'] = Real[0]
    #resultados['SEM_DC_LSTM'] = PREV_LSTM[0]
    #resultados['SEM_DC_RF'] = PREV_RF[0]
    #resultados['SEM_DC_ARV'] = PREV_ARV[0]
  
    #nome = 'resultados_SEM_DC_'+arquivos[cont]
    #resultados.to_excel(nome)
    
    cont = cont+3